In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Aula de Aprendizado de Máquina
# Código para resolver Atividade 1 sobre Regressão Linear

import pandas as pd

df = pd.read_csv("/content/air_traffic.csv")

df.head(10)

FileNotFoundError: [Errno 2] No such file or directory: '/content/air_traffic.csv'

In [ ]:
from google.colab import drive


In [ ]:

import pandas as pd

# Selecionar as colunas de Dom_Pax até ASM
df_slice = df[["Year","Month","Dom_Pax", "Int_Pax", "Pax", "Dom_Flt", "Int_Flt", "Flt","Dom_RPM", "Int_RPM", "RPM","Dom_ASM", "Int_ASM", "ASM"]].astype(str)

# Remover vírgulas e converter para inteiro
df_slice = df_slice.applymap(lambda x: int(x.replace(",", "")))

# Atualizar o DataFrame original com as colunas modificadas
df.update(df_slice)

# Exibir as primeiras 10 linhas do DataFrame
df.head(10)


In [ ]:

import matplotlib.pyplot as plt

# Plot the time series for Pax
plt.figure(figsize=(16, 6))
plt.plot(df['Year'] + df['Month'] / 12, df['Pax'], label='Pax')
plt.xlabel('Tempo')
plt.ylabel('Número de passageiros')
plt.title('Série temporal de passageiros')
plt.xticks(range(df['Year'].min(), df['Year'].max() + 1))
plt.grid(True)
plt.show()

# Plot the time series for Flt
plt.figure(figsize=(16, 6))
plt.plot(df['Year'] + df['Month'] / 12, df['Flt'], label='Flt')
plt.xlabel('Tempo')
plt.ylabel('Número de voos')
plt.title('Série temporal de voos')
plt.xticks(range(df['Year'].min(), df['Year'].max() + 1))
plt.grid(True)
plt.show()


In [ ]:

import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt

# Plot the time series for Flt
plt.figure(figsize=(16, 6))

#df1 = df[(df['Year'] <= 2007) & (df['Month'] < 9) & (df['Year'] == 2008)]
df2 = df[((df['Year'] == 2008) & (df['Month'] >= 9)) | (df['Year'] > 2008)]

# Plot the first period (Jan/2003 to Ago/2008)
plt.plot(df['Year'] + df['Month'] / 12, df['Flt'], color='blue', label='Jan/2003 a Ago/2008')

# Plot the second period (Set/2008 to Dez/2019)
plt.plot(df2['Year']+ df2['Month'] / 12, df2['Flt'], color='red', label='Set/2008 a Dez/2019')

# Plot the third period (Jan/2020 to Set/2023)
plt.plot(df['Year'][df['Year'] >= 2020] + df['Month'][df['Year']  >= 2020] / 12, df['Flt'][df['Year']  >= 2020], color='green', label='Jan/2020 a Set/2023')

plt.xlabel('Tempo')
plt.ylabel('Número de voos')
plt.title('Série temporal de voos')
plt.xticks(range(df['Year'].min(), df['Year'].max() + 1))
plt.grid(True)
plt.legend()
plt.show()


In [ ]:

import pandas as pd

# Calculate the mean of flights for each year in the period Jan/2003 to Ago/2008
df1 = df[(df['Year'] <= 2007) | ((df['Month'] < 9) & (df['Year'] == 2008))]
mean_flights_2003_2008 = df1.groupby('Year')['Flt'].mean()

# Calculate the mean of flights for each year in the period Set/2008 to Dez/2019
df2 = df[((df['Year'] == 2008) & (df['Month'] >= 9)) | (df['Year'] > 2008)]
mean_flights_2008_2019 = df2.groupby('Year')['Flt'].mean()

# Calculate the mean of flights for each year in the period Jan/2020 to Set/2023
df3 = df[df['Year'] >= 2020]
mean_flights_2020_2023 = df3.groupby('Year')['Flt'].mean()

# Print the results
print("Mean flights per year (Jan/2003 to Ago/2008):")
print(mean_flights_2003_2008)

print("\nMean flights per year (Set/2008 to Dez/2019):")
print(mean_flights_2008_2019)

print("\nMean flights per year (Jan/2020 to Set/2023):")
print(mean_flights_2020_2023)


In [ ]:

# Filtrar dados de 2003 e 2004
df_2003_2004 = df[(df["Year"] == 2003) | (df["Year"] == 2004)]

# Criar lista de meses e anos para o eixo x
x_axis_labels = []
for year in [2003, 2004]:
  for month in range(1, 13):
    x_axis_labels.append(f"{month:02d}/{year}")

# Plotar gráfico
plt.figure(figsize=(15, 5))
plt.plot(x_axis_labels, df_2003_2004["Flt"], label="Flt")
plt.xlabel("Tempo")
plt.ylabel("Número de Voos")
plt.title("Série Temporal de Voos (2003-2004)")
plt.legend()
plt.xticks(rotation=90)
plt.show()


In [ ]:

df_train_val = df[df['Year'] <= 2019]
df_test = df[df['Year'] >= 2020]

df_train_val.head(10)

In [ ]:
df_test.tail(10)

In [ ]:

porcentagem_treinamento = 0.8

train_size = int(len(df_train_val) * porcentagem_treinamento)
df_train = df_train_val[:train_size]
df_val = df_train_val[train_size:]

print(df_train.tail(10))
print(df_val.head(10))
print(len(df_val))

# Concatena DataFrames
#df_val = pd.concat([df_train.iloc[-2:], df_val])
#print(df_val.head(10))

In [ ]:

import numpy as np
from sklearn.linear_model import LinearRegression

# Definir o número de meses anteriores a serem considerados
#k = 24

rmse_train_vector = []
rmse_val_vector = []

for k in range(1, 25):

  # Criar matrizes X e y para treinamento
  X_train = []
  y_train = []
  for i in range(k, len(df_train)):
    X_train.append(df_train['Flt'][i-k:i])
    y_train.append(df_train['Flt'][i])
  X_train = np.array(X_train)
  y_train = np.array(y_train)

  #print(X_train)
  #print(y_train)

  # Treinar o modelo de regressão linear
  model = LinearRegression()
  model.fit(X_train, y_train)

  # Calcular o RMSE do conjunto de treinamento
  y_pred_train = model.predict(X_train)
  rmse_train = np.sqrt(np.mean((y_train - y_pred_train)**2))

  #print('RMSE do conjunto de treinamento:', rmse_train)
  #print(y_pred_train)

  #-----------------------------------------------------------------------------
  ## Outra forma de predição

  ## Treinar o modelo de regressão linear
  #w = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train

  ## Calcular o RMSE do conjunto de treinamento
  #y_pred_train = X_train @ w
  #rmse_train = np.sqrt(np.mean((y_train - y_pred_train)**2))

  #print("RMSE do conjunto de treinamento:", rmse_train)
  #-----------------------------------------------------------------------------

  # prompt: Preciso predizer o o conjunto de validação de df_val agora

  Ajuste_dados_previos_val = True
  # Ajusta o vetor de validação para o modelo
  if (Ajuste_dados_previos_val):
    df_val_modelo = pd.concat([df_train.iloc[-k:], df_val])
  else:
    df_val_modelo = df_val

  # Criar matrizes X e y para validação
  X_val = []
  y_val = []
  for i in range(k, len(df_val_modelo)):
    X_val.append(df_val_modelo['Flt'][i-k:i])
  X_val = np.array(X_val)

  y_val.append(df_val_modelo['Flt'][k:])
  y_val = np.array(y_val)

  # Predizer o conjunto de validação
  y_pred_val = model.predict(X_val)

  # Calcular o RMSE do conjunto de validação
  rmse_val = np.sqrt(np.mean((y_val - y_pred_val)**2))

  if k==1:
    best_model = model
    best_rmse_val = rmse_val
    best_k = k
  elif best_rmse_val > rmse_val:
    best_model = model
    best_rmse_val = rmse_val
    best_k = k

  #print('RMSE do conjunto de validação:', rmse_val)
  rmse_train_vector.append(rmse_train)
  rmse_val_vector.append(rmse_val)


In [ ]:
print(rmse_val_vector)
print(min(rmse_val_vector))
print(best_k)

In [ ]:

print(rmse_val_vector)

# Plot the RMSE values for each k
plt.figure(figsize=(10, 6))
plt.plot(range(1, 25), rmse_val_vector, label="RMSE Validação")
plt.plot(range(1, 25), rmse_train_vector, label="RMSE Treinamento")
plt.xlabel("k")
plt.ylabel("RMSE")
plt.title("RMSE vs. k")
plt.legend()
plt.show()


In [ ]:

k = best_k
Ajuste_dados_previos_teste = True

# Ajusta o vetor de validação para o modelo
if (Ajuste_dados_previos_teste):
  df_test_modelo = pd.concat([df_val.iloc[-k:], df_test])
else:
  df_test_modelo = df_test

# Criar matrizes X e y para teste
X_test = []
y_test = []

for i in range(k, len(df_test_modelo)):
  X_test.append(df_test_modelo['Flt'][i-k:i])

X_test = np.array(X_test)

y_test.append(df_test_modelo['Flt'][k:])
y_test = np.array(y_test)

# Predizer o conjunto de teste
y_pred_test = best_model.predict(X_test)

# Calcular o RMSE do conjunto de teste
rmse_test = np.sqrt(np.mean((y_test - y_pred_test)**2))

print('RMSE do conjunto de teste:', rmse_test)

x_axis_labels = []
for year in range(2020, 2024):
  for month in range(1, 13):
    if (year == 2023) & (month == 10): break
    x_axis_labels.append(f"{month:02d}/{year}")

print(len(y_test[0]), len(x_axis_labels))

# Plot the predicted values vs. the actual values
plt.figure(figsize=(10, 6))
plt.plot(x_axis_labels, y_test[0], label="Real")
plt.plot(x_axis_labels, y_pred_test, label="Predito")
plt.xlabel("Tempo")
plt.ylabel("Número de Voos")
plt.title("Predição de Voos (2020-2022)")
plt.legend()
plt.xticks(rotation=90)
plt.show()

#print(y_test)
#print(y_pred_test)

In [ ]:

# Calculate the mean absolute percentage error (MAPE) for the test set
mape_test = np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100

# Print the RMSE and MAPE for the test set
print("RMSE for the test set:", rmse_test)
print("MAPE for the test set:", mape_test)


In [ ]:

df_new_test = df_test[df_test['Year'] >= 2022]

Ajuste_dados_previos_teste = True

# Ajusta o vetor de validação para o modelo
if (Ajuste_dados_previos_teste):
  df_new_test_modelo = pd.concat([df_val.iloc[-k:], df_new_test])
else:
  df_new_test_modelo = df_new_test

# Criar matrizes X e y para teste
X_test = []
y_test = []

for i in range(k, len(df_new_test_modelo)):
  X_test.append(df_new_test_modelo['Flt'][i-k:i])

X_test = np.array(X_test)

y_test.append(df_new_test_modelo['Flt'][k:])
y_test = np.array(y_test)

# Predizer o conjunto de teste
y_pred_test = best_model.predict(X_test)

# Calcular o RMSE do conjunto de teste
rmse_test = np.sqrt(np.mean((y_test - y_pred_test)**2))

# Calculate the mean absolute percentage error (MAPE) for the test set
mape_test = np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100

x_axis_labels = []
for year in range(2022, 2024):
  for month in range(1, 13):
    if (year == 2023) & (month == 10): break
    x_axis_labels.append(f"{month:02d}/{year}")

# Plot the real and predicted values for the last two years (2022 and 2023)
plt.figure(figsize=(10, 6))
plt.plot(x_axis_labels, y_test[0], label="Real")
plt.plot(x_axis_labels, y_pred_test, label="Predito")
plt.xlabel("Tempo")
plt.ylabel("Número de Voos")
plt.title("Predição de Voos (2022-2023)")
plt.legend()
plt.xticks(rotation=90)
plt.show()

# Print the RMSE and MAPE for the test set
print("RMSE for the test set:", rmse_test)
print("MAPE for the test set:", mape_test)


In [ ]:
df_train = df[df['Year'] <= 2019]
df_val = df[(df['Year'] > 2019) & (df['Year'] < 2022)]
df_test = df[df['Year'] >= 2022]

In [ ]:
rmse_train_vector = []
rmse_val_vector = []

for k in range(1, 25):

  # Criar matrizes X e y para treinamento
  X_train = []
  y_train = []
  for i in range(k, len(df_train)):
    X_train.append(df_train['Flt'][i-k:i])
    y_train.append(df_train['Flt'][i])
  X_train = np.array(X_train)
  y_train = np.array(y_train)

  #print(X_train)
  #print(y_train)

  # Treinar o modelo de regressão linear
  new_model = LinearRegression()
  new_model.fit(X_train, y_train)

  # Calcular o RMSE do conjunto de treinamento
  y_pred_train = new_model.predict(X_train)
  rmse_train = np.sqrt(np.mean((y_train - y_pred_train)**2))

  Ajuste_dados_previos_val = True
  # Ajusta o vetor de validação para o modelo
  if (Ajuste_dados_previos_val):
    df_val_modelo = pd.concat([df_train.iloc[-k:], df_val])
  else:
    df_val_modelo = df_val

  # Criar matrizes X e y para validação
  X_val = []
  y_val = []
  for i in range(k, len(df_val_modelo)):
    X_val.append(df_val_modelo['Flt'][i-k:i])
  X_val = np.array(X_val)

  y_val.append(df_val_modelo['Flt'][k:])
  y_val = np.array(y_val)

  # Predizer o conjunto de validação
  y_pred_val = new_model.predict(X_val)

  # Calcular o RMSE do conjunto de validação
  rmse_val = np.sqrt(np.mean((y_val - y_pred_val)**2))

  if k==1:
    best_model = new_model
    best_rmse_val = rmse_val
    best_k = k
  elif best_rmse_val > rmse_val:
    best_model = new_model
    best_rmse_val = rmse_val
    best_k = k

  #print('RMSE do conjunto de validação:', rmse_val)
  rmse_train_vector.append(rmse_train)
  rmse_val_vector.append(rmse_val)

In [ ]:
print(rmse_val_vector)
print(min(rmse_val_vector))
print(best_k)

In [ ]:
print(rmse_val_vector)

# Plot the RMSE values for each k
plt.figure(figsize=(10, 6))
plt.plot(range(1, 25), rmse_val_vector, label="RMSE Validação")
plt.plot(range(1, 25), rmse_train_vector, label="RMSE Treinamento")
plt.xlabel("k")
plt.ylabel("RMSE")
plt.title("RMSE vs. k")
plt.legend()
plt.show()

In [ ]:
k = best_k

Ajuste_dados_previos_teste = True

# Ajusta o vetor de validação para o modelo
if (Ajuste_dados_previos_teste):
  df_test_modelo = pd.concat([df_val.iloc[-k:], df_test])
else:
  df_test_modelo = df_test

# Criar matrizes X e y para teste
X_test = []
y_test = []

for i in range(k, len(df_test_modelo)):
  X_test.append(df_test_modelo['Flt'][i-k:i])

X_test = np.array(X_test)

y_test.append(df_test_modelo['Flt'][k:])
y_test = np.array(y_test)

# Predizer o conjunto de teste
y_pred_test = best_model.predict(X_test)

# Calcular o RMSE do conjunto de teste
rmse_test = np.sqrt(np.mean((y_test - y_pred_test)**2))

print('RMSE do conjunto de teste:', rmse_test)

x_axis_labels = []
for year in range(2022, 2024):
  for month in range(1, 13):
    if (year == 2023) & (month == 10): break
    x_axis_labels.append(f"{month:02d}/{year}")

print(len(y_test[0]), len(x_axis_labels))

# Plot the predicted values vs. the actual values
plt.figure(figsize=(10, 6))
plt.plot(x_axis_labels, y_test[0], label="Real")
plt.plot(x_axis_labels, y_pred_test, label="Predito")
plt.xlabel("Tempo")
plt.ylabel("Número de Voos")
plt.title("Predição de Voos (2022-2023)")
plt.legend()
plt.xticks(rotation=90)
plt.show()

#print(y_test)
#print(y_pred_test)

In [ ]:
# Calculate the mean absolute percentage error (MAPE) for the test set
mape_test = np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100

# Print the RMSE and MAPE for the test set
print("RMSE for the test set:", rmse_test)
print("MAPE for the test set:", mape_test)